<a href="https://colab.research.google.com/github/byunsy/equal-weight-index-fund/blob/main/Equal_Weight_S%26P.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Equal-Weight S&P 500 Index Fund

## Import Necessary Modules

In [3]:
import numpy as np 
import pandas as pd 
import requests 
import math 

In [4]:
!pip install -q finance-datareader
import FinanceDataReader as fdr

## Attain S&P Stock Listing

In [5]:
sp500 = fdr.StockListing('S&P500')
sp500.head(10)

,Symbol,Name,Sector,Industry
0,MMM,3M Company,Industrials,Industrial Conglomerates
1,ABT,Abbott Laboratories,Health Care,Health Care Equipment
2,ABBV,AbbVie Inc.,Health Care,Pharmaceuticals
3,ABMD,ABIOMED Inc,Health Care,Health Care Equipment
4,ACN,Accenture plc,Information Technology,IT Consulting & Other Services
5,ATVI,Activision Blizzard,Communication Services,Interactive Home Entertainment
6,ADBE,Adobe Inc.,Information Technology,Application Software
7,AMD,Advanced Micro Devices Inc,Information Technology,Semiconductors
8,AAP,Advance Auto Parts,Consumer Discretionary,Automotive Retail
9,AES,AES Corp,Utilities,Independent Power Producers & Energy Traders


In [6]:
sp500['Symbol']

0       MMM
1       ABT
2      ABBV
3      ABMD
4       ACN
       ... 
500     YUM
501    ZBRA
502     ZBH
503    ZION
504     ZTS
Name: Symbol, Length: 505, dtype: object

## API Call

We first need to have a test api token to use IEX Cloud APIs (This will remain private). You can receive sandbox Text APIs from the IEX Cloud API website. 

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
from iex_api import IEX_CLOUD_API_TOKEN

In [17]:
# To take an example of what we get from IEX Cloud, we will take Microsoft
symbol='MSFT'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
ms_data = requests.get(api_url).json()

ms_data

{'avgTotalVolume': 25874593,
 'calculationPrice': 'close',
 'change': 0.64,
 'changePercent': 0.00289,
 'close': None,
 'closeSource': 'aciliffo',
 'closeTime': None,
 'companyName': 'Microsoft Corporation',
 'delayedPrice': None,
 'delayedPriceTime': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPrice': None,
 'extendedPriceTime': None,
 'high': None,
 'highSource': None,
 'highTime': None,
 'iexAskPrice': 0,
 'iexAskSize': 0,
 'iexBidPrice': 0,
 'iexBidSize': 0,
 'iexClose': 229,
 'iexCloseTime': 1642175775251,
 'iexLastUpdated': 1653404570424,
 'iexMarketPercent': 0.0012525032924742827,
 'iexOpen': 226.58,
 'iexOpenTime': 1655670244636,
 'iexRealtimePrice': 237,
 'iexRealtimeSize': 34,
 'iexVolume': 172,
 'isUSMarketOpen': False,
 'lastTradeTime': 1658274761699,
 'latestPrice': 230.29,
 'latestSource': 'Close',
 'latestTime': 'January 21, 2021',
 'latestUpdate': 1645357931495,
 'latestVolume': None,
 'low': None,
 'lowSource': None,
 'lowTime': None,
 'mar

We can now get specific information about our data using indices. 

In [18]:
print("LATEST PRICE:", ms_data['latestPrice'])
print("MARKET CAPITALIZATION:", ms_data['marketCap'])

LATEST PRICE: 230.29
MARKET CAPITALIZATION: 1731458494926


## Data Preprocessing


We will now transfer our attained data into a data frame. 

In [ ]:
df_columns = ['Symbol', 'Latest Price','Market Capitalization', 'Number Of Shares to Purchase']
df = pd.DataFrame(columns=df_columns)

for symbol in sp500['Symbol']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    df = df.append(pd.Series([symbol, data['latestPrice'], 
                              data['marketCap'], 'N/A'], index=df_columns), 
                              ignore_index = True)
df   

However we realize this takes quite a long time to process. This is because API calls can be time-consuming. One way to cope with this is to use batch API calls.

In [ ]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [ ]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []

for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()

    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'], 
                                                   data[symbol]['quote']['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe